In [1]:
import json
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
num_robots = 1
sampling_rate = 2
dataset = 'bathymetry'
budget = True
benchmark = 0

# Available benchmarks
benchmark = ['Adaptive', 'Discrete', 'Continuous'][benchmark]

# Adaptive Methods
if benchmark=='Adaptive':
    discrete = False
    colors = ['C2', 'C3', 'C0', 'C1']
    methods = ['Adaptive-SGP', 
               'Adaptive-CMA-ES', 
               'SGP', 
               'CMA-ES']
# Discrete Methods
elif benchmark=='Discrete':
    discrete = True
    colors = ['C2', 'C3', 'C0']
    methods = ['Discrete-SGP',
               'Greedy-MI', 
               'Greedy-SGP']
# Continuous Methods
elif benchmark=='Continuous':
    discrete = False
    colors = ['C2', 'C3', 'C0']
    methods = ['SGP', 
               'CMA-ES']
    if num_robots == 1 and not budget:
        methods.append('BO')

if budget:
    flag = '_B'
else:
    flag = ''

if discrete:
    flag = '_D'

rmse = defaultdict(list)
rmse_std = defaultdict(list)

param_time = defaultdict(list)
param_time_std = defaultdict(list)

ipp_time = defaultdict(list)
ipp_time_std = defaultdict(list)

filename = f'results/{dataset}_{num_robots}R_{sampling_rate}S{flag}.json'
save_filename = filename[:-5] + '-{}.pdf'
results = json.load(open(filename, 'r'))

xrange = np.array(list(results.keys())).astype(int)
for num_sensors in results.keys():
    for method in  results[num_sensors].keys():
        for metric in results[num_sensors][method].keys():
            if metric=='RMSE':
                rmse[method].append(np.mean(results[num_sensors][method][metric]))
                rmse_std[method].append(np.std(results[num_sensors][method][metric]))
            elif metric=='ParamTime':
                data = np.array(results[num_sensors][method][metric])
                if 'Adaptive' in method:
                    data /= int(num_sensors)
                param_time[method].append(np.mean(data))
                param_time_std[method].append(np.std(data))
            elif metric=='IPPTime':
                data = np.array(results[num_sensors][method][metric])
                if 'Adaptive' in method:
                    data /= int(num_sensors)
                ipp_time[method].append(np.mean(data))
                ipp_time_std[method].append(np.std(data))

if benchmark == 'Continuous' and num_robots == 1 and not budget:
    filename = f'results/{dataset}_{num_robots}R_{sampling_rate}S_D.json'
    save_filename = filename[:-5] + '-{}.pdf'
    results = json.load(open(filename, 'r'))

    xrange = np.array(list(results.keys())).astype(int)
    for num_sensors in results.keys():
        for method in  results[num_sensors].keys():
            for metric in results[num_sensors][method].keys():
                if metric=='RMSE':
                    rmse[method].append(np.mean(results[num_sensors][method][metric]))
                    rmse_std[method].append(np.std(results[num_sensors][method][metric]))
                elif metric=='ParamTime':
                    data = np.array(results[num_sensors][method][metric])
                    if 'Adaptive' in method:
                        data /= int(num_sensors)
                    param_time[method].append(np.mean(data))
                    param_time_std[method].append(np.std(data))
                elif metric=='IPPTime':
                    data = np.array(results[num_sensors][method][metric])
                    if 'Adaptive' in method:
                        data /= int(num_sensors)
                    ipp_time[method].append(np.mean(data))
                    ipp_time_std[method].append(np.std(data))

plt.figure()
for i, key in enumerate(methods):
    xrange_ = xrange[:len(rmse[key])]
    label = key.strip().replace('Adaptive-SGP', 'Adaptive-SGP (Ours)')
    if benchmark=='Adaptive':
        label = label.replace('SGP', 'SGP-IPP')
        label = label.replace('Adaptive-CMA-ES', 'CIPP')
        label = label.replace('CMA-ES', 'Non-Adaptive-CIPP')
    if discrete:
        label = label.replace('Discrete-SGP', 'Discrete-SGP (Ours)')
    if benchmark=='Continuous' and num_robots==1:
        label = label.replace('SGP', 'SGP (Ours)')
    plt.plot(xrange_, rmse[key], label=label, c=colors[i])
    plt.fill_between(xrange_,
                     np.array(rmse[key])+rmse_std[key], 
                     np.array(rmse[key])-rmse_std[key], 
                     alpha=0.2,
                     color=colors[i])
plt.legend(loc='upper right')
plt.xlabel(f"Number of {'Waypoints' if budget else 'Sensors'}")
plt.ylabel("RMSE")
plt.savefig(f'{dataset}_{num_robots}R_{sampling_rate}S{flag}-RMSE.pdf', bbox_inches='tight', dpi=500)
plt.show()

plt.figure()
for i, key in enumerate(methods):
    xrange_ = xrange[:len(ipp_time[key])]
    label = key.strip().replace('Adaptive-SGP', 'Adaptive-SGP (Ours)')
    if benchmark=='Adaptive':
        label = label.replace('SGP', 'SGP-IPP')
        label = label.replace('Adaptive-CMA-ES', 'CIPP')
        label = label.replace('CMA-ES', 'Non-Adaptive-CIPP')
    if discrete:
        label = label.replace('Discrete-SGP', 'Discrete-SGP (Ours)')
    if benchmark=='Continuous' and num_robots==1:
        label = label.replace('SGP', 'SGP (Ours)')
    plt.plot(xrange_, ipp_time[key], label=label, c=colors[i])
    plt.fill_between(xrange_,
                     np.array(ipp_time[key])+ipp_time_std[key], 
                     np.array(ipp_time[key])-ipp_time_std[key], 
                     alpha=0.2,
                     color=colors[i])
plt.legend(loc='upper left')
plt.xlabel(f"Number of {'Waypoints' if budget else 'Sensors'}")
plt.ylabel("Runtime (Seconds)")
plt.savefig(f'{dataset}_{num_robots}R_{sampling_rate}S{flag}-IPP.pdf', bbox_inches='tight', dpi=500)
#plt.show()

In [ ]:
# Times speedup
np.array(ipp_time['Adaptive-CMA-ES']).mean()/np.array(ipp_time['Adaptive-SGP']).mean()